# Compare cover metrics

Comparing cover metrics with field estimates

In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray

data_dir = Path('../data')

In [2]:
# Read in user defined storey limits
field_estimates = pd.read_csv(data_dir / "outputs/lidar_assessed_storey_limits.csv")

# Drop missing rows
field_estimates = field_estimates.dropna(subset=['la_ground_limit'])
field_estimates = field_estimates.dropna(subset=['veg_ground_total', 'veg_understorey_total', 'veg_midstorey_total', 'veg_upperstorey_total'], how='all')
field_estimates = field_estimates.set_index('site_plot_id')
field_estimates = field_estimates[['veg_ground_total', 'veg_understorey_total', 'veg_midstorey_total', 'veg_upperstorey_total']]
field_estimates = field_estimates.rename(columns={
    "veg_ground_total": "field_groundstorey",
    "veg_understorey_total": "field_understorey",
    "veg_midstorey_total": "field_midstorey",
    "veg_upperstorey_total": "field_upperstorey"
})

field_estimates

,field_groundstorey,field_understorey,field_midstorey,field_upperstorey
site_plot_id,,,,
AGG_O_01_P1,9.0,3.0,NaN,20.0
AGG_O_01_P2,6.0,3.0,NaN,9.0
AGG_O_01_P3,3.0,3.0,NaN,17.0
AGG_O_01_P4,9.0,18.0,19.0,2.0
AGG_O_01_P5,12.0,10.0,17.0,2.0
...,...,...,...,...
ULY_Y_96_P1,31.0,20.0,7.0,0.0
ULY_Y_96_P2,35.0,39.0,7.0,0.0
ULY_Y_96_P3,29.0,26.0,3.0,3.0


In [3]:
metrics_dir = data_dir / 'outputs/plots/metrics/x1-y1-z1/net_cdf'

def read_plot_metrics(plot_id: str):
    metrics = xr.open_dataset(metrics_dir / f"{plot_id}_with_cover.nc", decode_coords='all')
    metrics.load()
    metrics.close()
    return metrics

def get_cover_metrics(row: pd.Series):
    plot_id = row.name
    metrics_ds = read_plot_metrics(plot_id)

    storeys = ['ground', 'under', 'mid', 'upper']
    results = {}

    for storey in storeys:
        metric_suffixes= ['rel_density', 'rel_density_w', 'capture', 'capture_w']
        for m in metric_suffixes:
            metric_name = f'{storey}storey_{m}'
            if metric_name in metrics_ds:
                results[metric_name] = metrics_ds[metric_name].mean().item() * 100
            else:
                results[metric_name] = np.nan

    results = pd.Series(results)
    return pd.concat([row, results])

In [4]:
field_estimates_and_cover_metrics = field_estimates.apply(get_cover_metrics, axis=1)
field_estimates_and_cover_metrics = field_estimates_and_cover_metrics.fillna(0)
field_estimates_and_cover_metrics

,field_groundstorey,field_understorey,field_midstorey,field_upperstorey,groundstorey_rel_density,groundstorey_rel_density_w,groundstorey_capture,groundstorey_capture_w,understorey_rel_density,understorey_rel_density_w,understorey_capture,understorey_capture_w,midstorey_rel_density,midstorey_rel_density_w,midstorey_capture,midstorey_capture_w,upperstorey_rel_density,upperstorey_rel_density_w,upperstorey_capture,upperstorey_capture_w
site_plot_id,,,,,,,,,,,,,,,,,,,,
AGG_O_01_P1,9.0,3.0,0.0,20.0,19.583480,20.040613,47.107531,47.589575,8.299044,6.886897,15.808909,13.233880,0.000000,0.000000,0.000000,0.000000,48.069095,48.251447,48.069095,48.251447
AGG_O_01_P2,6.0,3.0,0.0,9.0,14.771845,14.871917,40.017746,40.261297,8.889967,7.257913,16.962559,14.209936,0.000000,0.000000,0.000000,0.000000,49.035656,49.759529,49.035656,49.759529
AGG_O_01_P3,3.0,3.0,0.0,17.0,16.320254,16.731635,46.729173,47.359706,10.122964,8.838800,21.044011,18.724461,0.000000,0.000000,0.000000,0.000000,53.028951,53.491853,53.028951,53.491853
AGG_O_01_P4,9.0,18.0,19.0,2.0,14.221226,15.177263,62.130701,62.735738,36.440707,35.066259,61.614193,60.162441,24.440203,24.712230,29.238302,29.404582,16.152455,16.071951,16.152455,16.071951
AGG_O_01_P5,12.0,10.0,17.0,2.0,8.542778,8.995911,46.745395,47.536790,25.217956,23.967261,55.052000,53.892033,36.707002,37.011001,46.777609,47.324608,20.039120,20.406860,20.039120,20.406860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P1,31.0,20.0,7.0,0.0,35.431852,37.447591,62.537015,62.893920,9.370167,8.618875,19.436907,18.116012,40.175624,38.771108,40.574282,39.185590,1.000904,0.914550,1.000904,0.914550
ULY_Y_96_P2,35.0,39.0,7.0,0.0,49.867921,52.351544,75.031218,75.260539,13.266476,12.533446,19.201015,17.973027,18.126360,15.978534,19.382333,17.046309,3.562411,3.416375,3.562411,3.416375
ULY_Y_96_P3,29.0,26.0,3.0,3.0,49.467737,52.261583,79.341715,79.882609,22.613940,21.958494,28.132139,26.659556,14.396437,12.189175,14.552274,12.286679,0.659513,0.495731,0.659513,0.495731


In [5]:
field_estimates_and_cover_metrics.to_csv(data_dir / "outputs/field_estimates_and_cover_metrics.csv")

## Comparing weighted and unweighted cover metrics

In [6]:
from scipy.stats import pearsonr, spearmanr, linregress

In [8]:
storeys = ["groundstorey", "understorey", "midstorey", "upperstorey"]
lidar_metrics = ["rel_density", "rel_density_w", "capture", "capture_w"]


def maybe_scale_lidar_to_percent(field, lidar):
    """Auto-scale LiDAR (×100) when field looks like % and LiDAR looks like proportions."""
    f_med = np.nanmedian(field)
    l_med = np.nanmedian(lidar)
    if np.isfinite(f_med) and np.isfinite(l_med) and (f_med > 1 and l_med <= 1.0):
        return lidar * 100.0, True
    return lidar, False


rows = []
df = field_estimates_and_cover_metrics

for s in storeys:
    field_col = f"field_{s}"
    field_vals = df[field_col].astype(float).to_numpy()

    for mname in lidar_metrics:
        lidar_col = f"{s}_{mname}"
        lidar_vals = df[lidar_col].astype(float).to_numpy()

        mask = np.isfinite(field_vals) & np.isfinite(lidar_vals)
        x = lidar_vals[mask]
        y = field_vals[mask]
        n = int(mask.sum())

        sr = spearmanr(x, y)

        slope, intercept, r_value, p_value, std_err = linregress(x, y)

        rmse = float(np.sqrt(np.mean((x - y) ** 2)))
        mae  = float(np.mean(np.abs(x - y)))
        bias = float(np.mean(x - y))

        rows.append(
            {
                "storey": s,
                "metric": mname,
                "n": n,
                "pearson_r": r_value,
                "pearson_p": p_value,
                "spearman_rho": float(sr.correlation),
                "spearman_p": float(sr.pvalue),
                "slope": slope,
                "intercept": intercept,
                "r2": r_value * r_value,
                "RMSE": rmse,
                "MAE": mae,
                "bias": bias

            }
        )

summary = pd.DataFrame(rows)
summary = summary.set_index(["storey", "metric"]).sort_index()

summary

n  pearson_r     pearson_p  spearman_rho  \
storey       metric                                                      
groundstorey capture        308   0.466092  5.132955e-18      0.496440   
             capture_w      308   0.465832  5.384790e-18      0.494930   
             rel_density    308   0.289830  2.250363e-07      0.391529   
             rel_density_w  308   0.279354  6.284984e-07      0.383096   
midstorey    capture        308   0.764211  3.111392e-60      0.860912   
             capture_w      308   0.765553  1.456564e-60      0.860971   
             rel_density    308   0.787821  2.251617e-66      0.853565   
             rel_density_w  308   0.789365  8.386289e-67      0.852686   
understorey  capture        308   0.514375  3.337502e-22      0.542085   
             capture_w      308   0.517944  1.537011e-22      0.549131   
             rel_density    308   0.492301  3.311091e-20      0.504337   
             rel_density_w  308   0.494868  1.973024e-20      0.511733   
upperstorey  capture        308   0.653572  6.623069e-39      0.636231   
             capture_w      308   0.653410  7.009801e-39      0.636971   
             rel_density    308   0.653572  6.623069e-39      0.636231   
             rel_density_w  308   0.653410  7.009801e-39      0.636971   

                              spearman_p     slope  intercept        r2  \
storey       metric                                                       
groundstorey capture        1.434165e-20  0.451324   0.726673  0.217241   
             capture_w      1.948607e-20  0.445137   0.901975  0.216999   
             rel_density    1.003488e-12  0.447088  14.843388  0.084002   
             rel_density_w  3.321013e-12  0.403768  15.227015  0.078039   
midstorey    capture        8.212734e-92  0.425299  -0.354557  0.584018   
             capture_w      7.736454e-92  0.416722  -0.227077  0.586072   
             rel_density    1.190831e-88  0.600120  -0.015782  0.620662   
             rel_density_w  2.770454e-88  0.594878   0.189326  0.623098   
understorey  capture        6.338218e-25  0.344276   5.453981  0.264581   
             capture_w      1.172210e-25  0.338910   5.959790  0.268266   
             rel_density    2.813693e-21  0.445618  10.612730  0.242360   
             rel_density_w  5.889539e-22  0.443902  10.974596  0.244895   
upperstorey  capture        2.385339e-36  0.347671   2.942610  0.427157   
             capture_w      1.869797e-36  0.336467   3.139950  0.426945   
             rel_density    2.385339e-36  0.347671   2.942610  0.427157   
             rel_density_w  1.869797e-36  0.336467   3.139950  0.426945   

                                 RMSE        MAE       bias  
storey       metric                                          
groundstorey capture        28.419984  24.996453  22.459235  
             capture_w      28.651663  25.186377  22.652817  
             rel_density    19.050579  13.318770  -8.715163  
             rel_density_w  19.284948  13.557104  -8.476134  
midstorey    capture        27.498203  17.573516  16.794670  
             capture_w      28.128790  17.849435  17.077541  
             rel_density    16.486698   9.640319   7.896835  
             rel_density_w  16.512028   9.550420   7.725713  
understorey  capture        26.331537  21.023592  18.015012  
             capture_w      25.948234  20.319673  17.089330  
             rel_density    15.790207  11.049814  -1.701118  
             rel_density_w  15.934735  11.118408  -2.454170  
upperstorey  capture        28.782077  23.964028  22.449187  
             capture_w      29.753578  24.723383  23.158822  
             rel_density    28.782077  23.964028  22.449187  
             rel_density_w  29.753578  24.723383  23.158822

In [9]:
mae_compare = (
    summary["MAE"]
    .unstack("metric")  # columns = capture, capture_w, rel_density, rel_density_w
    .assign(
        capture_diff = lambda df: df["capture_w"] - df["capture"],
        rel_density_diff = lambda df: df["rel_density_w"] - df["rel_density"]
    )
)
mae_compare

metric,capture,capture_w,rel_density,rel_density_w,capture_diff,rel_density_diff
storey,,,,,,
groundstorey,24.996453,25.186377,13.318770,13.557104,0.189924,0.238334
midstorey,17.573516,17.849435,9.640319,9.550420,0.275919,-0.089900
understorey,21.023592,20.319673,11.049814,11.118408,-0.703919,0.068593
upperstorey,23.964028,24.723383,23.964028,24.723383,0.759355,0.759355


In [10]:
df = field_estimates_and_cover_metrics
for storey in ["groundstorey", "understorey", "midstorey", "upperstorey"]:
    mae_diff = (df[f"{storey}_rel_density"] - df[f"{storey}_rel_density_w"]).abs().mean()
    print(f"{storey}: MAE between rel_density and rel_density_w = {mae_diff:.3f}")

for storey in ["groundstorey", "understorey", "midstorey", "upperstorey"]:
    mae_diff = (df[f"{storey}_capture"] - df[f"{storey}_capture_w"]).abs().mean()
    print(f"{storey}: MAE between capture and capture_w = {mae_diff:.3f}")

groundstorey: MAE between rel_density and rel_density_w = 0.579
understorey: MAE between rel_density and rel_density_w = 0.924
midstorey: MAE between rel_density and rel_density_w = 0.540
upperstorey: MAE between rel_density and rel_density_w = 1.551
groundstorey: MAE between capture and capture_w = 0.285
understorey: MAE between capture and capture_w = 1.120
midstorey: MAE between capture and capture_w = 0.711
upperstorey: MAE between capture and capture_w = 1.551
